In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
PATH = "../data/Q2"

In [3]:
f = open(PATH + "_data_full.txt", "r")
s = f.read()
s = s.split('\n')
s[:5]

['RELATIVE SYSCALL(args) \t\t = return',
 'dtrace: 114 dynamic variable drops with non-empty dirty list',
 ' 5397140 psynch_cvsignal(0x7FB56F8AE4C0, 0x2AB2F0002AB3000, 0x2AB2F00)\t\t = 257 0',
 ' 5397144 gettimeofday(0x7000152B67C8, 0x0, 0x0)\t\t = 0 0',
 '106783647 psynch_cvsignal(0x7FB56F8AA8C0, 0x144C0000144C100, 0x144BF00)\t\t = 257 0']

In [4]:
s = [line.strip() for line in s]
s = s[2:]
s[:3]

['5397140 psynch_cvsignal(0x7FB56F8AE4C0, 0x2AB2F0002AB3000, 0x2AB2F00)\t\t = 257 0',
 '5397144 gettimeofday(0x7000152B67C8, 0x0, 0x0)\t\t = 0 0',
 '106783647 psynch_cvsignal(0x7FB56F8AA8C0, 0x144C0000144C100, 0x144BF00)\t\t = 257 0']

In [5]:
s = [re.split('\(|\)|,\s+|\s+', line)[:5] for line in s]

In [6]:
df = pd.DataFrame(columns=['relative_timestamp', 'activity', 'path', 'p2', 'p3'], data=s)
df.drop(['p2', 'p3'], axis=1, inplace=True)
df.index.name = 'pd_timestamp'
df = df.astype("string")

In [7]:
df = df[df["path"].str.contains("/Comnata")][df["path"].str.contains(".kt")]
df["case_id"] = df["path"].str.extract(r"(nmbr\d+)")[0].str.extract(r"(\d+)")
df["activity_file"] = df["activity"] + " | " + df["path"].str.extract(r"([A-Za-z]+\.kt)")[0]
df = df[df['activity_file'].notna()]

/var/folders/hg/5ywdl7v542z8hcr7kg05v66h0000gn/T/ipykernel_15608/799959598.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[df["path"].str.contains("/Comnata")][df["path"].str.contains(".kt")]


In [8]:
df.head()

,relative_timestamp,activity,path,case_id,activity_file
pd_timestamp,,,,,
90662,976626,stat64,"""/Users/vakosta/Developer/IdeaProjects/Comnata...",2,stat64 | gradle.kt
90686,976739,stat64,"""/Users/vakosta/Developer/IdeaProjects/Comnata...",2,stat64 | gradle.kt
117003,2035349,open,"""/Users/vakosta/Developer/IdeaProjects/Comnata...",2,open | MainServiceApplication.kt
230578,271729,lstat,"""/Users/vakosta/Developer/IdeaProjects/Comnata...",4,lstat | WebsocketEventListener.kt
230579,4518584,lstat,"""/Users/vakosta/Developer/IdeaProjects/Comnata...",4,lstat | RoomRepository.kt


In [9]:
df.to_csv(PATH + "_data_filtered.csv", sep=',')